In [1]:
import pandas as pd

l_test = []
l_test.append(None)
l_test

[None]

In [12]:
import requests
def get_journey_time(from_station, to_station="940GZZLUAGL"):
    url = f"https://api.tfl.gov.uk/Journey/JourneyResults/{from_station}/to/{to_station}?date=20250710&time=0900&timeIs=Departing&journeyPreference=LeastTime&accessibilityPreference=NoRequirements&walkingSpeed=Average&cyclePreference=None"
    # url = f"https://api.tfl.gov.uk/Journey/JourneyResults/{from_station}/to/{to_station}"
    
    params = {}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        journey = data['journeys'][0]
        duration = journey['duration']
        return duration
    else:
        print(f"Error fetching journey from {from_station} to {to_station}")
        print(response)
        return None
    
get_journey_time("9400ZZDLLIM")

34

In [10]:
import pandas as pd
cols = ["ATCOCode", "CommonName"]
df_stops = pd.read_csv("/Users/sqwu/Downloads/Stops.csv")
df_stops_sub = df_stops[(df_stops["ParentLocalityName"] == "London") & (df_stops["ATCOCode"].str.len() == 11) & (df_stops["ATCOCode"].str[:3] == "940")]
df_stops_sub = df_stops_sub[cols].drop_duplicates()

df_stops_sub.head()
df_stops_sub.to_csv("../files/mapping/naptan_src2.csv")

/Users/sqwu/property_api/property_api/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (1,2,6,8,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
df_1 = pd.read_csv("../files/mapping/naptan_src1.csv")
df_2 = pd.read_csv("../files/mapping/naptan_src2.csv").drop(columns=["Unnamed: 0"], axis=1)

df_2 = df_2.rename(columns={"ATCOCode": "naptanID",
                            "CommonName": "commonName"})

In [24]:
df_final = pd.concat([df_1, df_2]).reset_index().drop(columns=["index"], axis=1)
print(df_final.shape)
df_final = df_final.drop_duplicates(subset=["naptanID"])
print(df_final.shape)

(545, 2)
(545, 2)


In [26]:
df_final.to_csv("../files/mapping/naptan.csv")

In [34]:
pd.set_option('display.max_rows', 100)
df_naptan = pd.read_csv("../files/mapping/naptan.csv")
df_naptan["commonName"] = df_naptan["commonName"].str.replace(" Underground Station","").str.replace(" Tram Stop","").str.replace(" DLR Station","")
df_naptan

,Unnamed: 0,naptanID,commonName
0,0,940GZZLUBST,Baker Street
1,1,940GZZLUCHX,Charing Cross
2,2,940GZZLUEAC,Elephant & Castle
3,3,940GZZLUEMB,Embankment
4,4,940GZZLUERB,Edgware Road (Bakerloo)
5,5,940GZZLUHAW,Harrow & Wealdstone
6,6,940GZZLUHSN,Harlesden
7,7,940GZZLUKEN,Kenton
8,8,940GZZLUKPK,Kilburn Park
9,9,940GZZLUKSL,Kensal Green


In [3]:
import pandas as pd
from thefuzz import process

# Your data
df = pd.DataFrame({'station_name': ['Pimlico', 'St James Park', 'Victoria']})
mapping_df = pd.DataFrame({
    'mapping_station': ['Pimlico Station', 'St. James\'s Park Station', 'Victoria Station'],
    'naptan_code': ['940GZZLUPCO', '940GZZLUSTP', '940GZZLUVIC']
})

# Function to fuzzy match and return best match + score
def match_station(station, choices, scorer_threshold=80):
    match, score, i = process.extractOne(station, choices)
    return match if score >= scorer_threshold else None

# Apply matching
df['matched_station'] = df['station_name'].apply(lambda x: match_station(x, mapping_df['mapping_station']))

# Merge on matched_station
merged_df = df.merge(mapping_df, left_on='matched_station', right_on='mapping_station', how='left')
merged_df

,station_name,matched_station,mapping_station,naptan_code
0,Pimlico,Pimlico Station,Pimlico Station,940GZZLUPCO
1,St James Park,St. James's Park Station,St. James's Park Station,940GZZLUSTP
2,Victoria,Victoria Station,Victoria Station,940GZZLUVIC


In [6]:
import os

# Get current working directory
print("Current directory:", os.getcwd())

# Change to a new directory
new_dir = "/Users/sqwu/property_api/property_api"
os.chdir(new_dir)

# Verify the change
print("New directory:", os.getcwd())

Current directory: /Users/sqwu/property_api/property_api/demo
New directory: /Users/sqwu/property_api/property_api


In [7]:
import pandas as pd
from sqlalchemy import create_engine
from src.database import db

database = db.SessionLocal()
data = database.query(db.FlatsToRent).all()
df = pd.DataFrame([task.__dict__ for task in data])

# Check for duplicates
duplicates = df[df.duplicated(['unique_id'], keep=False)]
print(f"Found {len(duplicates)} potential duplicates")
print(duplicates.head())

Found 0 potential duplicates
Empty DataFrame
Columns: [_sa_instance_state, rent, property_type, base, number_of_bathroom, link, address, unique_id, location, price, number_of_bedroom, description, run_time]
Index: []


In [3]:
data = [
  {
    "address": "Valencia Tower, London, EC1V",
    "location": "Islington",
    "price": 3600,
    "number_of_bedroom": 1,
    "description": "One-bedroom apartment in the highly anticipated 250 City Road development, EC1V. The property comprises an open plan kitchen living area, bedroom with built in storage and stylish bathroom.",
    "run_time": "2025-05-25",
    "property_type": "Flat",
    "unique_id": "1|1|2025-05-25|06:10:55.078496",
    "rent": "£3,600 pcm",
    "base": "pcm",
    "number_of_bathroom": 1,
    "link": "https://www.rightmove.co.uk/properties/134069828#/?channel=RES_LET"
  },
  {
    "address": "Charlotte Terrace, London, N1 0TS",
    "location": "Islington",
    "price": 1750,
    "number_of_bedroom": 1,
    "description": "Hauzoo are delighted to market this delightful newly refurbished 1 bedroom, 1 bathroom ground floor flat with private garden located in Islington.",
    "run_time": "2025-05-25",
    "property_type": "Flat",
    "unique_id": "1|2|2025-05-25|06:10:55.078496",
    "rent": "£1,750 pcm",
    "base": "pcm",
    "number_of_bathroom": 1,
    "link": "https://www.rightmove.co.uk/properties/162418898#/?channel=RES_LET"
  },]

import pandas as pd
df = pd.DataFrame(data)
df.to_json()

'{"address":{"0":"Valencia Tower, London, EC1V","1":"Charlotte Terrace, London, N1 0TS"},"location":{"0":"Islington","1":"Islington"},"price":{"0":3600,"1":1750},"number_of_bedroom":{"0":1,"1":1},"description":{"0":"One-bedroom apartment in the highly anticipated 250 City Road development, EC1V. The property comprises an open plan kitchen living area, bedroom with built in storage and stylish bathroom.","1":"Hauzoo are delighted to market this delightful newly refurbished 1 bedroom, 1 bathroom ground floor flat with private garden located in Islington."},"run_time":{"0":"2025-05-25","1":"2025-05-25"},"property_type":{"0":"Flat","1":"Flat"},"unique_id":{"0":"1|1|2025-05-25|06:10:55.078496","1":"1|2|2025-05-25|06:10:55.078496"},"rent":{"0":"\\u00a33,600 pcm","1":"\\u00a31,750 pcm"},"base":{"0":"pcm","1":"pcm"},"number_of_bathroom":{"0":1,"1":1},"link":{"0":"https:\\/\\/www.rightmove.co.uk\\/properties\\/134069828#\\/?channel=RES_LET","1":"https:\\/\\/www.rightmove.co.uk\\/properties\\/16

In [2]:
len(l_test)

1

In [4]:
import re

price_str = "£2,400 pcm"
match = re.search(r'\d[\d,]*', price_str)

if match:
    price = int(match.group(0).replace(',', ''))
    print(price)  # Output: 2400
else:
    price = None


2400


In [7]:
import time
import datetime

ts = time.time()
print(ts)
ct = datetime.datetime.now()
print("current time:", ct)

1747491545.5240908
current time: 2025-05-17 15:19:05.524587


In [9]:
import pandas as pd

df = pd.DataFrame({"col1":["a","a","b"],
                   "col2":[1,1,2]})

df.drop_duplicates()

,col1,col2
0,a,1
2,b,2


In [13]:

a =  False
not a

True

In [11]:
len(df)

3

In [10]:
ts = time.time()
ct = datetime.datetime.now()
print("current time:", str(ct).replace(" ","|"))

current time: 2025-05-17|15:52:02.520054
